# 1479. Sales by Day of the Week

## Table: Orders

| Column Name  | Type    |
|--------------|---------|
| order_id     | int     |
| customer_id  | int     |
| order_date   | date    | 
| item_id      | varchar |
| quantity     | int     |

`(order_id, item_id)` is the primary key (combination of columns with unique values) for this table. This table contains information on the orders placed. `order_date` is the date `item_id` was ordered by the customer with `customer_id`.

## Table: Items

| Column Name   | Type    |
|---------------|---------|
| item_id       | varchar |
| item_name     | varchar |
| item_category | varchar |

`item_id` is the primary key (column with unique values) for this table. `item_name` is the name of the item. `item_category` is the category of the item.

You are the business owner and would like to obtain a sales report for category items and the day of the week.

Write a solution to report how many units in each category have been ordered on each day of the week.

Return the result table ordered by category.

The result format is in the following example.

## Example 1:

**Input:**

Orders table:

| order_id | customer_id | order_date | item_id | quantity |
|----------|--------------|-------------|---------|----------|
| 1        | 1            | 2020-06-01  | 1       | 10       |
| 2        | 1            | 2020-06-08  | 2       | 10       |
| 3        | 2            | 2020-06-02  | 1       | 5        |
| 4        | 3            | 2020-06-03  | 3       | 5        |
| 5        | 4            | 2020-06-04  | 4       | 1        |
| 6        | 4            | 2020-06-05  | 5       | 5        |
| 7        | 5            | 2020-06-05  | 1       | 10       |
| 8        | 5           


In [1]:
import pandas as pd

orders_data = {
    'order_id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'customer_id': [1, 1, 2, 3, 4, 4, 5, 5, 5],
    'order_date': ['2020-06-01', '2020-06-08', '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-05', '2020-06-14', '2020-06-21'],
    'item_id': [1, 2, 1, 3, 4, 5, 1, 4, 3],
    'quantity': [10, 10, 5, 5, 1, 5, 10, 5, 5]
}

orders = pd.DataFrame(orders_data)
orders['order_date'] = pd.to_datetime(orders['order_date'])

items_data = {
    'item_id': [1, 2, 3, 4, 5, 6],
    'item_name': ['LC Alg. Book', 'LC DB. Book', 'LC SmarthPhone', 'LC Phone 2020', 'LC SmartGlass', 'LC T-Shirt XL'],
    'item_category': ['Book', 'Book', 'Phone', 'Phone', 'Glasses', 'T-Shirt']
}

items = pd.DataFrame(items_data)

In [2]:
def sales_by_day(orders: pd.DataFrame, items: pd.DataFrame) -> pd.DataFrame:
    orders['day'] = orders['order_date'].dt.day_name()
    days_list = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    merged_df = pd.merge(left=orders,right=items,on='item_id')
    merged_df = merged_df.groupby(['item_category','day']).sum('quantity').reset_index()
    pivot_df = merged_df.pivot(index='item_category', columns='day', values='quantity').reset_index().rename({'item_category':'Category'},axis=1)
    pivot_df.fillna(0,inplace=True)
    for day in days_list:
        if day not in pivot_df.columns:
            pivot_df[day] = len(pivot_df) * [0]
    for item in items['item_category'].unique():
        if item not in pivot_df['Category'].unique():
            sub_dict = {col:[0.0] for col in pivot_df.columns}
            sub_dict['Category'][0]=item
            pivot_df = pd.concat([pivot_df,pd.DataFrame(sub_dict)])

    return pivot_df[['Category']+days_list]

In [3]:
sales_by_day(orders=orders,items=items)

,Category,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Book,20.0,5.0,0.0,0.0,10.0,0.0,0.0
1,Glasses,0.0,0.0,0.0,0.0,5.0,0.0,0.0
2,Phone,0.0,0.0,5.0,1.0,0.0,0.0,10.0
0,T-Shirt,0.0,0.0,0.0,0.0,0.0,0.0,0.0
